# PROJEKT MODELU KLASYFIKACYJNEGO PRZEWIDYWANIA STANU MIESZKAŃ NA PODSTAWIE #

## Extra Trees Classifier ##

### maj 2025

In [1]:
import pandas as pd
import mlflow
from pycaret.classification import setup, pull, compare_models, plot_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import resample
import pymysql
from sqlalchemy import create_engine
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# tą komórkę uruchom jeżeli bazę danych bierzesz np. z DBeaver, oracle developer itp.

#username = 'root'
#password = '1234'
#host = '127.0.0.1'
#port = 3306  
#database = 'projekt1'
#engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

#df = pd.read_sql("SELECT * FROM saleflats", con=engine)

#engine.dispose()

In [3]:
MLFLOW_EXPERIMENT_NAME = 'Investoro_Stan_Mieszkania'
MLFLOW_TAGS = {'data': 'Investoro_stany', 'library': 'pycaret'}

mlflow.set_tracking_uri("http://localhost:5000")

In [4]:
# tę komórkę uruchom jeśli czerpiesz dane z pliku .csv
df = pd.read_csv('sale_2024_0.csv', sep=',')

In [5]:
df

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,88,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Mieszkanie o powierzchni 73m2 znajduje się na ...,73.00,766500.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,NaN,02TgImo1wm-Ra4zUQxoPBw==
1,99,NaN,19,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Antoniuk",20.0,61.0,1.0,1.0,922426.0,NaN,NaN,D-NUdVbtN3_kIW7ExgXkvA==
2,115,NaN,3,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bagnówka",20.0,61.0,1.0,1.0,922461.0,NaN,NaN,lmBSU1XiXLSLOq1yKSe2kA==
3,140,NaN,14,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,544000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Józefa Karola Puchal...",20.0,61.0,1.0,1.0,922410.0,NaN,43141.0,HWachf5gb6Vq-sFYH4YWnA==
4,145,NaN,3,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,459000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Dziesięciny",20.0,61.0,1.0,1.0,922410.0,NaN,4656.0,AWsmt4ZY0X7lNv-6fSCn9Q==
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
1133883,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
1133884,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
1133885,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [6]:
# pobranie próbki 10 losowych wierszy
df.sample(10)

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
170563,1220568,NaN,3,Moderno Łobzów | mieszkanie 3-pok. | 46,3-pokojowe mieszkanie numer 46 na parterze w I...,67.40,979000.0,950460.0,1480666.67,1019000.0,...,NaN,"Małopolskie, Kraków, Kraków-krowodrza, Krowodr...",12.0,61.0,2.0,9.0,950569.0,NaN,20060.0,4obBsqEwekeTTpUrBoG0-atLCFuALKtSKAVt1gPg8yU=
206113,1516177,NaN,19,Mieszkanie z tarasem 20km KRK,zapraszamy do zapoznania się z ofertą mieszkan...,56.00,499000.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków",12.0,61.0,1.0,1.0,950463.0,NaN,NaN,qvR_7D3xmrmHXbWMVPiMCOjdq9zykoRglLyNhCMTntA=
867854,3810169,NaN,19,"Karczew, Maczka 48,37 m², 2-pokojowe, bezpośre...",Mieszkanie w bardzo dobrej lokalizacji w Karcz...,48.37,505000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Otwocki, Karczew",14.0,17.0,4.0,4.0,NaN,NaN,NaN,XaDLF-LwBngh2END-1226TD3XjSU8KMxGYrRgZJdWOU=
191120,1391220,NaN,4,3-pokojowe mieszkanie na sprzedaż,ATUTY: - Gotowe do wprowadzenia - 400 metrów d...,59.00,650000.0,NaN,NaN,NaN,...,NaN,"Pomorskie, Gdański, Pruszcz Gdański, Ul. Bened...",22.0,4.0,1.0,1.0,934620.0,NaN,4485.0,aHL2hUeCF7apQx1vKM0sE69HKvZhXIinsDp7FqZAtfA=
314821,2168801,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Przedstawiam wyjątkowe mieszkanie w Radzyniu P...,48.40,260000.0,NaN,NaN,NaN,...,NaN,"Lubelskie, Radzyński, Radzyń Podlaski, Ul. Spó...",6.0,15.0,1.0,1.0,922277.0,NaN,20696.0,OtIQ7v3X-dults1ja5SkkvtiMRU9ErLfA6XOKdBw9jY=
864286,3800329,NaN,11,"Mieszkanie, Świętochłowice, 62 m²","Mieszkanie, Świętochłowice, 62 m² Ogłoszenioda...",62.00,249000.0,NaN,NaN,NaN,...,NaN,"Śląskie, Świętochłowice",24.0,76.0,NaN,NaN,NaN,NaN,NaN,D3pgmbeZhkjk7dROM4OzTubQxVp1jDeL4EC2z-A_Xv4=
490110,2867579,NaN,3,"2 Pokoje, Nowe Bloki, Świdnik",Mieszkanie składa się z przestronnego salonu z...,35.12,399000.0,NaN,NaN,415000.0,...,NaN,"Lubelskie, Świdnicki, Świdnik",6.0,17.0,1.0,1.0,957146.0,NaN,NaN,ukx4LaEyGB48fO3exR0YW_HeWU4K50Qo__6slT6OOS8=
735068,3450128,NaN,35,"Giełda Płocka - Sprzedam Mieszkanie, Słupno, 81m2",NA SPRZEDAŻ mieszkanie bezczynszowe o powierz...,81.00,790000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Płocki, Słupno",14.0,19.0,12.0,2.0,575901.0,NaN,NaN,t2U53opUe_3D7uHcqU6lZ4_bCM8lJ876moazm9h2OAo=
786257,3594032,NaN,3,Mieszkanie trzypokojowe do odbioru na już! | B...,"Oferta BEZPOŚREDNIO OD DEWELOPERA, BEZ PODATKU...",48.62,819000.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków, Kraków-podgórze, Ul. Rydl...",12.0,61.0,4.0,9.0,950960.0,NaN,19210.0,Z3JA6tTtZacuBD6zDIxJ6t9BBYQp3JUW_C-K3oFvgXA=
331565,2245962,NaN,19,"Legnica, woj. dolnośląskie. Sprzedam mieszkanie","Sprzedam kawalerkę, Legnica, ul. Astronomiczna...",34.60,199000.0,NaN,NaN,NaN,...,NaN,"Dolnośląskie, Legnica",2.0,62.0,NaN,NaN,NaN,NaN,NaN,-qGLTvLh5J_mLmS7Tha8WJShIpC99iUkqTPiCKTTxVM=


In [7]:
print(df.columns)

Index(['SaleId', 'OriginalId', 'PortalId', 'Title', 'Description', 'Area',
       'Price', 'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
       'PricePerSquareMeter', 'NumberOfRooms', 'BuiltYear', 'Type',
       'BuildingType', 'BuildingCondition', 'OfferFrom', 'Floor', 'Floors',
       'TypeOfMarket', 'OwnerType', 'DateAddedToDatabase', 'DateAdded',
       'DateLastModification', 'DateLastRaises', 'NewestDate', 'AvailableFrom',
       'Link', 'Phone', 'MainImage', 'OtherImages', 'NumberOfDuplicates',
       'NumberOfRaises', 'NumberOfModifications', 'IsDuplicatePriceLower',
       'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision', 'CommunityScore',
       'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
       'Location', 'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber',
       'KindNumber', 'RegionNumber', 'SubRegionNumber', 'StreetNumber',
       'EncryptedId'],
      dtype='object')


In [8]:
df['BuiltYear'] = pd.to_datetime(df['BuiltYear'], format='%Y', errors='coerce')

In [9]:
df_a = df.dropna(subset=['Description'])

In [10]:
df_a.isnull().sum()

SaleId                             0
OriginalId                   1125223
PortalId                           0
Title                              0
Description                        0
Area                            2077
Price                          53212
OfferPrice                   1043154
RealPriceAfterRenovation     1043154
OriginalPrice                1015114
PricePerSquareMeter            56195
NumberOfRooms                  13525
BuiltYear                     379005
Type                               0
BuildingType                  202788
BuildingCondition            1048333
OfferFrom                       5030
Floor                          56904
Floors                        499074
TypeOfMarket                   31709
OwnerType                     540943
DateAddedToDatabase                0
DateAdded                          0
DateLastModification          880033
DateLastRaises                827954
NewestDate                         0
AvailableFrom                1125223
L

In [11]:
df_b = df_a.dropna(subset=['Location'])

In [12]:
df_b.isnull().sum()

SaleId                             0
OriginalId                   1124683
PortalId                           0
Title                              0
Description                        0
Area                            2071
Price                          53153
OfferPrice                   1042614
RealPriceAfterRenovation     1042614
OriginalPrice                1014574
PricePerSquareMeter            56133
NumberOfRooms                  13494
BuiltYear                     378808
Type                               0
BuildingType                  202543
BuildingCondition            1047795
OfferFrom                       5030
Floor                          56828
Floors                        498541
TypeOfMarket                   31629
OwnerType                     540441
DateAddedToDatabase                0
DateAdded                          0
DateLastModification          879543
DateLastRaises                827486
NewestDate                         0
AvailableFrom                1124683
L

In [13]:
df_c = df_b.dropna(subset=['BuildingCondition'])

In [14]:
df_c.isnull().sum()

SaleId                           0
OriginalId                   76888
PortalId                         0
Title                            0
Description                      0
Area                            30
Price                         7036
OfferPrice                   31258
RealPriceAfterRenovation     31258
OriginalPrice                73440
PricePerSquareMeter           7138
NumberOfRooms                  165
BuiltYear                    19248
Type                             0
BuildingType                 15600
BuildingCondition                0
OfferFrom                        0
Floor                         1984
Floors                       13600
TypeOfMarket                   716
OwnerType                    30525
DateAddedToDatabase              0
DateAdded                        0
DateLastModification         68857
DateLastRaises               68330
NewestDate                       0
AvailableFrom                76888
Link                             0
Phone               

In [15]:
df_c

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
1037441,4236909,NaN,10,Mieszkanie na sprzedaż - 56 m² - 3 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,56.52,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF1I945vZrxCmGc1kx-MElW4=
1037442,4236910,NaN,10,Mieszkanie na sprzedaż - 39 m² - 3 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,39.44,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF2b3MhONsWHx4CM5o0osslE=
1037444,4236917,NaN,10,Mieszkanie na sprzedaż - 57 m² - 4 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,57.72,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF5P7_4V7ShzJmmF8PHTVfng=
1037449,4236925,NaN,10,Mieszkanie na sprzedaż - 34 m² - 2 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,34.69,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF-baRshu3omO9IJePLIRnwA=
1037450,4236928,NaN,10,Mieszkanie na sprzedaż - 37 m² - 2 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,37.03,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF4_bCM8lJ876moazm9h2OAo=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
1133883,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
1133884,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
1133885,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [16]:
df_c['Description'] = df_c['Description'].str.slice(0, 3000)

In [17]:
# Przygotuj wektorizer z limitem
vectorizer = CountVectorizer(max_features=1500)  # np. 1500 najczęstszych słów
X_bow = vectorizer.fit_transform(df_c["Description"])

# Konwertuj do DataFrame
df_bow = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Dołącz do oryginalnych danych (z wyjątkiem kolumny Description)
df_c = df_c.drop(columns=['Description']).reset_index(drop=True)
df_bow = df_bow.reset_index(drop=True)
df_prepared = pd.concat([df_c, df_bow], axis=1)

In [18]:
df_prepared.info

<bound method DataFrame.info of         SaleId  OriginalId  PortalId  \
0      4236909         NaN        10   
1      4236910         NaN        10   
2      4236917         NaN        10   
3      4236925         NaN        10   
4      4236928         NaN        10   
...        ...         ...       ...   
76883  4418401         NaN         2   
76884  4418402         NaN         2   
76885  4418404         NaN         2   
76886  4418405         NaN         2   
76887  4418406         NaN         2   

                                                   Title   Area  Price  \
0      Mieszkanie na sprzedaż - 56 m² - 3 pokoje Wars...  56.52    NaN   
1      Mieszkanie na sprzedaż - 39 m² - 3 pokoje Wars...  39.44    NaN   
2      Mieszkanie na sprzedaż - 57 m² - 4 pokoje Wars...  57.72    NaN   
3      Mieszkanie na sprzedaż - 34 m² - 2 pokoje Wars...  34.69    NaN   
4      Mieszkanie na sprzedaż - 37 m² - 2 pokoje Wars...  37.03    NaN   
...                                        

In [19]:
df_prepared[df_prepared.duplicated()]

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świeżo,świeżym,żabka,żadnych,że,życia,życie,życiem,żłobek,żłobki


In [20]:
df_prepared.nunique()

SaleId        76888
OriginalId        0
PortalId         34
Title         56154
Area           8156
              ...  
życia             8
życie             5
życiem            3
żłobek            5
żłobki            4
Length: 1550, dtype: int64

In [21]:
df_prepared.isnull()

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świeżo,świeżym,żabka,żadnych,że,życia,życie,życiem,żłobek,żłobki
0,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76883,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
76884,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
76885,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
76886,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [22]:
df_prepared.isnull().sum()

SaleId            0
OriginalId    76888
PortalId          0
Title             0
Area             30
              ...  
życia             0
życie             0
życiem            0
żłobek            0
żłobki            0
Length: 1550, dtype: int64

In [23]:
Q1 = df_prepared["Price"].quantile(0.25)
Q3 = df_prepared["Price"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prep_p = df_prepared[~((df_prepared["Price"] < lower_bound) | (df_prepared["Price"] > upper_bound))]

In [24]:
Q1 = df_prep_p["PricePerSquareMeter"].quantile(0.25)
Q3 = df_prep_p["PricePerSquareMeter"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prep_a = df_prep_p[~((df_prep_p["PricePerSquareMeter"] < lower_bound) | (df_prep_p["PricePerSquareMeter"] > upper_bound))]

In [25]:
Q1 = df_prep_a["Area"].quantile(0.25)
Q3 = df_prep_a["Area"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prepared2 = df_prep_a[~((df_prep_a["Area"] < lower_bound) | (df_prep_a["Area"] > upper_bound))]

In [26]:
df_prepared2["BuildingCondition"].value_counts(normalize=True)

BuildingCondition
DEVELOPER_STATE     0.475638
AFTER_RENOVATION    0.349426
GOOD                0.087849
FOR_RENOVATION      0.087087
Name: proportion, dtype: float64

In [27]:
df_prepared2.nunique()

SaleId        69597
OriginalId        0
PortalId         34
Title         50634
Area           6522
              ...  
życia             8
życie             5
życiem            3
żłobek            5
żłobki            4
Length: 1550, dtype: int64

In [28]:
unique_btype=df_prepared2['BuildingType'].unique()

unique_btype

array([nan, 'Apartamentowiec', 'Blok', 'Pozostałe', 'Kamienica'],
      dtype=object)

In [29]:
print(df_prepared2['BuildingCondition'].nunique())

4


In [30]:
df_prepared2.isnull().sum()

SaleId            0
OriginalId    69597
PortalId          0
Title             0
Area             29
              ...  
życia             0
życie             0
życiem            0
żłobek            0
żłobki            0
Length: 1550, dtype: int64

In [31]:
df_prepared3=df_prepared2.dropna(subset=['BuildingCondition'])

In [32]:
df_prepared3.loc[df_prepared3['TypeOfMarket'] == 'pierwotny', 'BuildingCondition'] = 'DEVELOPER_STATE'

In [33]:
df_prepared3['BuiltYear'] = pd.to_datetime(df_prepared3['BuiltYear'], format='%Y', errors='coerce')

In [34]:
df_prepared3.loc[df_prepared3['BuiltYear'].dt.year >= 2024, 'BuildingCondition'] = 'DEVELOPER_STATE'

In [35]:
df_prepared3 = df_prepared3[~((df_prepared3['Link'].str.contains('otodom', case=False, na=False)) & 
                              (df_prepared3['BuildingCondition'] == 'AFTER_RENOVATION'))]

In [36]:
unique_ctype=df_prepared3['BuildingCondition'].unique()

unique_ctype

array(['DEVELOPER_STATE', 'AFTER_RENOVATION', 'GOOD', 'FOR_RENOVATION'],
      dtype=object)

In [37]:
df_prepared3["BuildingCondition"].value_counts(normalize=True)

BuildingCondition
DEVELOPER_STATE     0.618903
AFTER_RENOVATION    0.178385
FOR_RENOVATION      0.107558
GOOD                0.095154
Name: proportion, dtype: float64

In [ ]:
#POMINĄĆ W TEJ CHWILI setup USTAWIONY JEST na SMOTE
# DODANIE ZBALANSOWANIA
# Sprawdź ile rekordów ma każda klasa
class_counts = df_prepared3['BuildingCondition'].value_counts()
min_count = class_counts.min()

# Tworzymy zbalansowane próbki dla każdej klasy
dfs = []
for condition in class_counts.index:
    df_condition = df_prepared3[df_prepared3['BuildingCondition'] == condition]
    df_condition_downsampled = resample(df_condition, 
                                        replace=False, 
                                        n_samples=min_count, 
                                        random_state=42)
    dfs.append(df_condition_downsampled)

# Łączymy wszystkie klasy w jeden zbalansowany DataFrame
df_balanced = pd.concat(dfs).reset_index(drop=True)

In [ ]:
# df_balanced["BuildingCondition"].value_counts(normalize=True)

In [38]:
df_prepared3

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świeżo,świeżym,żabka,żadnych,że,życia,życie,życiem,żłobek,żłobki
0,4236909,NaN,10,Mieszkanie na sprzedaż - 56 m² - 3 pokoje Wars...,56.52,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,1,1,0,0,0,0
1,4236910,NaN,10,Mieszkanie na sprzedaż - 39 m² - 3 pokoje Wars...,39.44,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,1,1,0,0,0,0
2,4236917,NaN,10,Mieszkanie na sprzedaż - 57 m² - 4 pokoje Wars...,57.72,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,1,1,0,0,0,0
3,4236925,NaN,10,Mieszkanie na sprzedaż - 34 m² - 2 pokoje Wars...,34.69,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,1,1,0,0,0,0
4,4236928,NaN,10,Mieszkanie na sprzedaż - 37 m² - 2 pokoje Wars...,37.03,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76883,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,59.99,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
76884,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,41.23,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
76885,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,49.09,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
76886,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,49.48,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [39]:
train_df = df_prepared3.sample(frac=0.9, random_state=42)
holdout_df = df_prepared3.drop(train_df.index)

In [40]:
df_prepared3['VoivodeshipNumber'] = df_prepared3['VoivodeshipNumber'].astype(str)
df_prepared3['CountyNumber'] = df_prepared3['CountyNumber'].astype(str)
df_prepared3['CommunityNumber'] = df_prepared3['CommunityNumber'].astype(str)
df_prepared3['KindNumber'] = df_prepared3['KindNumber'].astype(str)
df_prepared3['RegionNumber'] = df_prepared3['RegionNumber'].astype(str)
if 'StreetNumber' in df_prepared.columns:
    df_prepared3['StreetNumber'] = df_prepared3['StreetNumber'].astype(str)

In [ ]:
categorical_features_initial = [
    'BuildingType', 'Price', 'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom',
    'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber',
    'RegionNumber', 'Location', 'Description'
]
numeric_features_initial = [
    'Area', 'NumberOfRooms', 'Floor', 'Floors', 'CommunityScore'
]
date_features_initial = ['BuiltYear']

# Filtruj cechy, które faktycznie istnieją w df_price3
categorical_features_to_use = [col for col in categorical_features_initial if col in df_prepared3.columns]
numeric_features_to_use = [col for col in numeric_features_initial if col in df_prepared3.columns]
date_features_to_use = [col for col in date_features_initial if col in df_prepared3.columns]

# Definicja cech ignorowanych
ignore_features_list = [
    'SaleId', 'OriginalId', 'PortalId', 'Title', 'Description',
    'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
    'PricePerSquareMeter', 'DateAddedToDatabase', 'DateAdded',
    'DateLastModification', 'DateLastRaises', 'NewestDate',
    'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages',
    'NumberOfDuplicates', 'NumberOfRaises', 'NumberOfModifications',
    'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision',
    'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
    'SubRegionNumber', 'EncryptedId',
    'StreetNumber' # Ignorujemy, bo może mieć zbyt wiele unikalnych wartości
]
# Filtruj ignorowane cechy, aby upewnić się, że są w DataFrame
ignore_features_list = [col for col in ignore_features_list if col in df_prepared3.columns]

transformed_exp = setup(
    data=df_prepared,
    target='BuildingCondition',
    verbose=True, # Ustaw na True dla debugowania setup
    session_id=123,
    log_experiment=True,
    experiment_name=MLFLOW_EXPERIMENT_NAME, 
    log_data=True, 
    log_plots=True,
    fix_imbalance=True,
    fix_imbalance_method="smote",
    categorical_features=categorical_features_to_use,
    numeric_features=numeric_features_to_use,
    date_features=date_features_to_use,
    # text_features=['Location'], # Usunięte na razie, przeniesione do categorical
    ignore_features=ignore_features_list,
    ordinal_features={'BuildingType': ['Pozostałe', 'Blok', 'Apartamentowiec', 'Kamienica'],
                     },
)
best_transformed_model = transformed_exp.compare_models()
transformed_metrics_df = pull()

,Description,Value
0,Session id,123
1,Target,BuildingCondition
2,Target type,Multiclass
3,Target mapping,"AFTER_RENOVATION: 0, DEVELOPER_STATE: 1, FOR_RENOVATION: 2, GOOD: 3"
4,Original data shape,"(76888, 1550)"
5,Transformed data shape,"(124067, 1560)"
6,Transformed train set shape,"(101000, 1560)"
7,Transformed test set shape,"(23067, 1560)"
8,Ignore features,31
9,Ordinal features,1


2025/05/22 18:22:31 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:23:37
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
transformed_exp.plot_model(best_transformed_model, plot='error')

In [ ]:
transformed_exp.plot_model(best_transformed_model, plot='feature')

In [ ]:
transformed_exp.dataset.head()

In [ ]:
transformed_exp.dataset_transformed.head()

In [ ]:
tuned_model = transformed_exp.tune_model(best_transformed_model, n_iter=3, optimize='F1')

In [ ]:
best_model = transformed_exp.compare_models([best_transformed_model, tuned_model])

In [ ]:
transformed_exp.predict_model(best_model)

In [ ]:
transformed_exp.plot_model(best_model, plot='error')

In [ ]:
transformed_exp.plot_model(best_model, plot='feature')

In [ ]:
best_tuned_model = transformed_exp.tune_model(
    best_model,
    optimize="F1",
    choose_better=True,
    fold=5,
)

In [ ]:
transformed_exp.predict_model(best_tuned_model).head(10)

In [ ]:
transformed_exp.plot_model(best_tuned_model, plot='error')

In [ ]:
transformed_exp.plot_model(best_tuned_model, plot='feature')

In [ ]:
best_final_model = transformed_exp.finalize_model(best_tuned_model)
best_final_model

In [ ]:
transformed_exp.save_model(best_final_model, "model_STAN_full", verbose=False);

In [ ]:
transformed_exp.predict_model(best_final_model, data=holdout_df)
predict_holdout_df = transformed_exp.pull()

In [ ]:
mlflow_exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
runs_df = mlflow.search_runs([mlflow_exp.experiment_id], order_by=["start_time DESC"])
run_id = runs_df['run_id'].values[0]
metrics = predict_holdout_df.drop(columns=["Model"]).to_dict(orient='records')[0]

with mlflow.start_run(run_id=run_id, nested=True):
    for key, value in metrics.items():
        mlflow.log_metric(key, value)

In [ ]:
#predictions = transformed_exp.predict_model(best_final_model)
holdout_predictions = transformed_exp.predict_model(best_final_model) # Zapisz wynik predykcji
display(holdout_predictions.head()) # Wyświetl kilka pierwszych predykcji

In [ ]:
#predicted_column = predict_holdout_df['prediction_label']
#print(predicted_column)
holdout_metrics_df = transformed_exp.pull() # Teraz pobierz metryki z tej predykcji
display(holdout_metrics_df)
predicted_column = holdout_predictions['prediction_label'] # Jeśli chcesz samą kolumnę predykcji
print(predicted_column)

In [ ]:
df_last=holdout_predictions[['prediction_label','BuildingCondition', ]]

In [ ]:
df_last.to_csv('2024_stan_full.csv')

## Confusion Matrix (Macierz pomyłek)

Confusion Matrix to tabela, która przedstawia liczbę poprawnych i niepoprawnych klasyfikacji dla każdej klasy. Ułatwia analizę, gdzie model popełnia błędy.

|               | Predicted Positive | Predicted Negative |
|---------------|--------------------|--------------------|
| Actual Positive | True Positive (TP)  | False Negative (FN) |
| Actual Negative | False Positive (FP) | True Negative (TN)  |
 
Analiza macierzy pomyłek pozwala zrozumieć, które klasy są mylone przez model oraz jaki jest balans między różnymi rodzajami błędów.

<span style="color:red">Idealny model miałby same wartości na przekątnej (TP i TN) oraz zera poza nią.</span>

In [ ]:
predicted_column = holdout_predictions['prediction_label']
print(predicted_column)

In [ ]:
df_last=holdout_predictions[['prediction_label','BuildingCondition', ]]

In [ ]:
df_last = df_last.reset_index()  
print(df_last.columns) 

In [ ]:
df_last.rename(columns={'index': 'SaleId'}, inplace=True)
print(df_last.columns) 

In [ ]:
df_last.to_csv('2024_stan_maz.csv')

In [ ]:
from pycaret.classification import load_model, predict_model

In [ ]:
best_final_model = load_model("model_STAN_maz")

In [ ]:
df_last

In [ ]:
# 1. Wczytaj dane
df_raw = pd.read_csv("2024_stan_maz.csv")

# 2. Zrób tymczasową kopię do predykcji — z placeholderami
df_predict = df_raw.copy()
df_predict["BuildingCondition"] = df_predict["BuildingCondition"].fillna(" ")

In [ ]:
# 1. Wyciągnij kolumny, które model zna
expected_cols = best_final_model.feature_names_in_

#  Napraw BuiltYear
df_raw['BuiltYear'] = pd.to_datetime(df_raw['BuiltYear'], errors='coerce')

# 2. Funkcja dopasowująca df_raw
def align_dataframe(df_raw, expected_cols):
    # Usuń 'BuildingCondition' z listy expected_cols
    expected_cols = [col for col in expected_cols if col != 'BuildingCondition']

    # Dodaj brakujące kolumny
    for col in expected_cols:
        if col not in df_raw.columns:
            df_raw[col] = 0  # albo NaN

    # Usuń BuildingCondition jeśli jest
    if 'BuildingCondition' in df_raw.columns:
        df_raw = df_raw.drop(columns=['BuildingCondition'])
    
    # Ustaw kolejność kolumn
    df_aligned = df_raw[expected_cols]

    return df_aligned

# 3. Użycie
df_aligned = align_dataframe(df_raw, expected_cols)

# 4. Predykcja
predictions = predict_model(best_final_model, data=df_aligned)

In [ ]:
result = df_aligned.copy()
result['Prediction_State'] = predictions['prediction_label']

# 5. Wyświetl
print(result)

In [ ]:
# 1. Skopiuj oryginalne dane
final_result = df_last.copy()

# 2. Usuń starą kolumnę BuildingCondition
if 'BuildingCondition' in final_result.columns:
    final_result = final_result.drop(columns=['BuildingCondition'])

# 3. Dodaj nową kolumnę z predykcją
final_result['Predicted_State'] = predict_holdout_df['prediction_label']

# 4. Wyświetl efekt
final_result

In [ ]:
# 1. Wczytaj dane z pliku
df_raw = pd.read_csv("sale_2024_0.csv")

# 2. Dodaj kolumnę z predykcją do oryginalnego df_raw
df_raw['Predicted_State'] = predict_holdout_df['prediction_label']

# 3. Wyświetl efekt
df_raw

In [ ]:
df_raw = df_raw[~((df_raw['Link'].str.contains('otodom', case=False, na=False)) & 
                              (df_raw['BuildingCondition'] == 'AFTER_RENOVATION'))]

In [ ]:
df_raw["Predicted_State"].value_counts(normalize=True)

In [ ]:
df_raw

In [ ]:
df_raw.to_csv('0_sale_2024_stan_final.csv')

In [ ]:
df1=pd.read_csv("0_sale_2024_stan_final.csv")

In [ ]:
df2=df1.sample(10)

In [ ]:
df2

In [ ]:
df2.to_csv('01_sample10.csv')